<a href="https://colab.research.google.com/github/DanielYzuel/Algoritmos-de-Optimizacion-VIU/blob/main/Daniel_Yzuel_Vaquerizo_Trabajo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario

Nombre y Apellidos: Daniel Yzuel Vaquerizo  

Url Colab: https://colab.research.google.com/drive/1k_5Z-kpyQvRgHeL1_JiSlpbI5GRpb2PK?usp=sharing

Url GitHub: https://github.com/DanielYzuel/Algoritmos-de-Optimizacion-VIU

Problema 1: Organizar sesiones de Doblaje


Descripción del problema:

 Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

1. Número de actores: 10
2. Número de tomas : 30
3. Actores/Tomas : https://bit.ly/36D8IuK

- 1 indica que el actor participa en la toma
- 0 en caso contrario

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools as it
import random

In [4]:
# Leemos los Datos
sheet = "1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0"
sheet_name = "Hoja1"
url = f"https://docs.google.com/spreadsheets/d/{sheet}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url, header = 1, index_col = 0, usecols = list(range(0, 11)), engine = "python", nrows = 30).add_prefix("Actor ")
df.index.name = 'Toma'

# Datos
n = df.shape[0]     # n = Número de tomas
m = df.shape[1]     # m = Número de actores
print(f"Número de tomas: {n}, Número de actores: {m}\n")

# Dataset
print('Actores requeridos (1: participa, 0: no participa)')
df['Número de actores'] = df.sum(axis = 1)
df.loc['Número de tomas'] = df.sum()
df.at['Número de tomas', 'Número de actores'] = np.nan
display(df)


Número de tomas: 30, Número de actores: 10

Actores requeridos (1: participa, 0: no participa)


,Actor 1,Actor 2,Actor 3,Actor 4,Actor 5,Actor 6,Actor 7,Actor 8,Actor 9,Actor 10,Número de actores
Toma,,,,,,,,,,,
1,1,1,1,1,1,0,0,0,0,0,5.0
2,0,0,1,1,1,0,0,0,0,0,3.0
3,0,1,0,0,1,0,1,0,0,0,3.0
4,1,1,0,0,0,0,1,1,0,0,4.0
5,0,1,0,1,0,0,0,1,0,0,3.0
6,1,1,0,1,1,0,0,0,0,0,4.0
7,1,1,0,1,1,0,0,0,0,0,4.0
8,1,1,0,0,0,1,0,0,0,0,3.0
9,1,1,0,1,0,0,0,0,0,0,3.0


Nos encontramos ante un problema en el cual, el número total de posibles ordenaciones de las tomas a lo largo de los días, se obtendrá calculando el producto del orden de las tomas por la distribución de las sesiones de grabación, que por identidad bionomial será:

In [5]:
n = 30
print(f'Posibilidades sin restricciones: \n30! = {math.factorial(n) * (2**(n-1))}')

Posibilidades sin restricciones: 
30! = 142406544757979162368320409970933760000000


Donde n es elnúmero total de tomas

Las restricciones son las siguientes:
1. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.
2. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben.
3. No es posible grabar más de 6 tomas por día.

De esta manera, el número total de posibilidades teniendo en cuenta las restricciones, vendría dado por:

In [6]:
n = 30
max = 6
P = 1
for j in range(0, max-1):
    P *= math.comb(n-max*j, max)

print(f"Posibilidades con restricciones: \nP = {P}")

Posibilidades con restricciones: 
P = 1370874167589326400


Donde "n" es el número de tomas y "max" es el número máximo de tomas que se pueden grabar en un día

La estructura de datos escogida para la resolución del problema va a ser:

- Datos entrada: Estructura de diccionario donde cada clave es una toma, y el valor de cada clave representa la participación o no-participación de los actores, tal y como se representa en el csv.
- Datos de salida: Un dataframe con 3 columnas que representen el número de la sesión, el coste de la sesión (Número de actores), y la última columna las tomas asignadas a dicha sesión de grabación.

In [8]:
# Eliminamos las filas y columnas innecesarias del .csv
df0 = df.drop(df.index[-1])
df0 = df0.drop(df0.columns[-1], axis = 1)

# Convertimos el df en un diccionario
d = {i + 1: list(fila) for i, fila in enumerate(df0.values)}

for k, v in d.items():
    print(f"{k}: {v}")

1: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
2: [0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
3: [0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
4: [1, 1, 0, 0, 0, 0, 1, 1, 0, 0]
5: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
6: [1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
7: [1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
8: [1, 1, 0, 0, 0, 1, 0, 0, 0, 0]
9: [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
10: [1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
11: [1, 1, 1, 0, 1, 0, 0, 1, 0, 0]
12: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
13: [1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
14: [1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
15: [1, 1, 0, 0, 0, 0, 1, 0, 0, 0]
16: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
17: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
18: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
19: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
20: [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
21: [0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
22: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
23: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
24: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
25: [1, 1, 0, 1, 0, 0, 0, 0, 0, 1]
26: [1, 0, 1, 0, 1, 0, 0, 0, 1, 0]
27: [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
28: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
29: [1, 0, 0, 0, 1, 1, 0, 0, 

Éste es un problema de minimización, ya que el objetivo es encontrar una planificación de las sesiones que minimice el costo total que suponen los actores al estudio de grabación.


En primer lugar debemos construir la función a minimizar (función objetivo), a la cual llamaremos "coste", que determinará el coste total asociado a una planificación de las sesiones (solución).


In [74]:
# Función objetivo
def coste(solucion, salario=100):
  coste_sesion = [sum(np.multiply((np.sum(sesion, axis = 0) != 0), salario)) for sesion in solucion]
  coste_total = sum(coste_sesion)
  return (coste_total,coste_sesion)

# Representación del planning
def repr_planning(solucion, salario = 100):
  coste_total, coste_sesion = coste(solucion, salario)
  sol = pd.DataFrame({'Sesión': pd.Series(range(len(coste_sesion))) + 1,
                      'Coste': coste_sesion})
  sol.loc['Total'] = (len(coste_sesion), coste_total)
  return sol

Tratemos de resolver el problema mediante un algoritmo de fuerza bruta

In [81]:
# Algoritmo por Fuerza Bruta
def fuerza_bruta(d, k = 6, num_perm = 1000):
  out_coste = float('inf')
  out = None
  permutaciones_tomas = it.permutations(list(d.keys()))

  for permutacion in it.islice(permutaciones_tomas, num_perm):
    d_inicial = np.array([d[i] for i in permutacion])
    solucion_actual = np.array_split(d_inicial, len(d_inicial) // k)
    coste_actual = repr_planning(solucion_actual)
    coste_actual['Tomas'] = pd.Series(np.array_split(permutacion, len(permutacion) // k))
    if coste_actual.at['Total', 'Coste'] < out_coste:
      out_coste = coste_actual.at['Total', 'Coste']
      out = coste_actual

  return out

In [82]:
# Probemos el algoritmo
fuerza_bruta(d)

,Sesión,Coste,Tomas
0,1,700,"[1, 2, 3, 4, 5, 6]"
1,2,800,"[7, 8, 9, 10, 11, 12]"
2,3,800,"[13, 14, 15, 16, 17, 18]"
3,4,800,"[19, 20, 21, 22, 23, 25]"
4,5,600,"[24, 26, 27, 28, 29, 30]"
Total,5,3700,NaN


Como podemos observar, las tomas están prácticamente ordenadas ascendentemente, ya que el algoritmo no ha tenido suficientes iteraciones como para llegar a un mínimo global.
Esto se debe a que la complejidad del algoritmo de fuerza bruta, al probar todas las combinaciones (permutaciones) posibles, es de O(n!).
Al ser n=30, esto daría del orden de 2.65e+32, lo cual lo hace un método inviable computacionalmente hablando.

Vamos a diseñar un algoritmo de búsqueda local, basado en una estrategia heurística que trata de mejorar de manera iterativa una solución inicial hasta llegar a un óptimo local (o hasta que se alcance el número de iteraciones máximas).

In [75]:
# Algoritmo de búsqueda local
def busqueda_local(d, k=6, max_iter=10000, intentos_sin_mejora=500):

    # Inicializamos el problema con una solución aleatoria
    tomas_iniciales = list(d.keys())
    random.shuffle(tomas_iniciales)
    solucion_actual = [tomas_iniciales[i:i+k] for i in range(0, len(tomas_iniciales), k)]
    matrices_actual = [np.array([d[toma] for toma in sesion]) for sesion in solucion_actual]

    costo_actual = coste(matrices_actual)[0]

    # Construimos el cuerpo de la función
    iter_sin_mejora = 0
    for _ in range(max_iter):
        if iter_sin_mejora >= intentos_sin_mejora:
            break

        idx_sesion1, idx_sesion2 = random.sample(range(len(solucion_actual)), 2)
        sesion1, sesion2 = solucion_actual[idx_sesion1][:], solucion_actual[idx_sesion2][:]
        toma1, toma2 = random.choice(sesion1), random.choice(sesion2)

        sesion1.remove(toma1)
        sesion2.remove(toma2)
        sesion1.append(toma2)
        sesion2.append(toma1)

        matrices_temp = matrices_actual[:]
        matrices_temp[idx_sesion1] = np.array([d[toma] for toma in sesion1])
        matrices_temp[idx_sesion2] = np.array([d[toma] for toma in sesion2])

        nuevo_costo = coste(matrices_temp)[0]
        if nuevo_costo < costo_actual:
            costo_actual = nuevo_costo
            matrices_actual = matrices_temp[:]
            solucion_actual[idx_sesion1] = sesion1
            solucion_actual[idx_sesion2] = sesion2
            iter_sin_mejora = 0
        else:
            iter_sin_mejora += 1

    out = repr_planning(matrices_actual)
    out['Tomas'] = pd.Series(solucion_actual)
    return out

In [76]:
# Probemos el algoritmo de búsqueda local
busqueda_local(d)

,Sesión,Coste,Tomas
0,1,600,"[21, 18, 23, 5, 12, 14]"
1,2,600,"[15, 9, 7, 2, 13, 28]"
2,3,500,"[6, 17, 22, 20, 19, 1]"
3,4,600,"[29, 30, 25, 27, 8, 16]"
4,5,800,"[24, 4, 3, 10, 26, 11]"
Total,5,3100,NaN


En este caso la complejidad es del orden de O(t n k), siendo t el número máximo de iteraciones, n el número de tomas y k el número de tomas por sesioón

Vemos que el algoritmo de búsqueda local supone una mejora sustancial con respecto del algoritmo de fuerza bruta ya que, suponiendo un salario de 100€ por actor y sesión, el estudio de grabación se ahorraría 600€.
No obstante, si la capacidad computacional no fuera un problema, el algoritmo de fuerza bruta siempre encontraría el mínimo global, mientras que el de búsqueda local es posible que nunca llegue a encontrarlo.
Teniendo en cuenta que este problema es bastante "simple" en comparación con los problemas reales sobre los que se implementan estos algoritmos, la búsqueda local va a ser siempre la opción más viable.

Como el algoritmo alcanza mínimos locales (no globales), una forma de mejorarlo sería construir otra función que ejecutara la función de busqueda_local un determinado número de veces, y guardara el mejor resultado, es decir, el mejor mínimo local.

Los recursos utilizados han sido principalmente el material de clase y la documentación de las distintas librerías empleadas.